<a href="https://colab.research.google.com/github/kamini2015/Complaint-Management-System/blob/master/Group_D_Code_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mercedes-Benz-Greener- Manufacturing


In [2]:
# Step 1: Import the required libraries
# Step 1.1: Linear algebra
import numpy as np
# Step 1.2: Data processing
import pandas as pd
from sklearn import preprocessing
# Step 1.3: For dimensionality reduction
from sklearn.decomposition import PCA
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [3]:
#  Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Update file paths to point to the location of your data on Google Drive
data_folder_path = '/content/drive/My Drive/Mercedes-Benz/'


In [6]:
# Step 2: Read the data from train.csv
train_df = pd.read_csv(data_folder_path + 'train.csv')

In [7]:
# Step 2.1: Let us understand the data
# Step 2.2: Print few rows and see how the data looks like
print(train_df.head())

   ID       y  X0 X1  X2 X3 X4 X5 X6 X8  ...  X375  X376  X377  X378  X379  \
0   0  130.81   k  v  at  a  d  u  j  o  ...     0     0     1     0     0   
1   6   88.53   k  t  av  e  d  y  l  o  ...     1     0     0     0     0   
2   7   76.26  az  w   n  c  d  x  j  x  ...     0     0     0     0     0   
3   9   80.62  az  t   n  f  d  x  l  e  ...     0     0     0     0     0   
4  13   78.02  az  v   n  f  d  h  d  n  ...     0     0     0     0     0   

   X380  X382  X383  X384  X385  
0     0     0     0     0     0  
1     0     0     0     0     0  
2     0     1     0     0     0  
3     0     0     0     0     0  
4     0     0     0     0     0  

[5 rows x 378 columns]


In [8]:
# Step 3: Collect the Y values into an array
# Step 3.1: Separate the y from the data as we will use this to learn
# as the prediction output
y_train = train_df['y']
X_train = train_df.drop(['ID', 'y'], axis=1)

In [9]:
# Step 4: Understand the data types we have
# Step 4.1: Iterate through all the columns which have X in the name
for col in X_train.columns:
    print(col)

X0
X1
X2
X3
X4
X5
X6
X8
X10
X11
X12
X13
X14
X15
X16
X17
X18
X19
X20
X21
X22
X23
X24
X26
X27
X28
X29
X30
X31
X32
X33
X34
X35
X36
X37
X38
X39
X40
X41
X42
X43
X44
X45
X46
X47
X48
X49
X50
X51
X52
X53
X54
X55
X56
X57
X58
X59
X60
X61
X62
X63
X64
X65
X66
X67
X68
X69
X70
X71
X73
X74
X75
X76
X77
X78
X79
X80
X81
X82
X83
X84
X85
X86
X87
X88
X89
X90
X91
X92
X93
X94
X95
X96
X97
X98
X99
X100
X101
X102
X103
X104
X105
X106
X107
X108
X109
X110
X111
X112
X113
X114
X115
X116
X117
X118
X119
X120
X122
X123
X124
X125
X126
X127
X128
X129
X130
X131
X132
X133
X134
X135
X136
X137
X138
X139
X140
X141
X142
X143
X144
X145
X146
X147
X148
X150
X151
X152
X153
X154
X155
X156
X157
X158
X159
X160
X161
X162
X163
X164
X165
X166
X167
X168
X169
X170
X171
X172
X173
X174
X175
X176
X177
X178
X179
X180
X181
X182
X183
X184
X185
X186
X187
X189
X190
X191
X192
X194
X195
X196
X197
X198
X199
X200
X201
X202
X203
X204
X205
X206
X207
X208
X209
X210
X211
X212
X213
X214
X215
X216
X217
X218
X219
X220
X221
X222
X223
X224
X225
X226
X227
X228

In [10]:
# Step 5: Count the data in each of the columns
for col in X_train.columns:
    print(f"Column: {col}, Unique Values: {X_train[col].nunique()}")


Column: X0, Unique Values: 47
Column: X1, Unique Values: 27
Column: X2, Unique Values: 44
Column: X3, Unique Values: 7
Column: X4, Unique Values: 4
Column: X5, Unique Values: 29
Column: X6, Unique Values: 12
Column: X8, Unique Values: 25
Column: X10, Unique Values: 2
Column: X11, Unique Values: 1
Column: X12, Unique Values: 2
Column: X13, Unique Values: 2
Column: X14, Unique Values: 2
Column: X15, Unique Values: 2
Column: X16, Unique Values: 2
Column: X17, Unique Values: 2
Column: X18, Unique Values: 2
Column: X19, Unique Values: 2
Column: X20, Unique Values: 2
Column: X21, Unique Values: 2
Column: X22, Unique Values: 2
Column: X23, Unique Values: 2
Column: X24, Unique Values: 2
Column: X26, Unique Values: 2
Column: X27, Unique Values: 2
Column: X28, Unique Values: 2
Column: X29, Unique Values: 2
Column: X30, Unique Values: 2
Column: X31, Unique Values: 2
Column: X32, Unique Values: 2
Column: X33, Unique Values: 2
Column: X34, Unique Values: 2
Column: X35, Unique Values: 2
Column: X36,

In [11]:
# Step 6: Read the test.csv data
test_df = pd.read_csv(data_folder_path + 'test.csv')

In [13]:
# Step 6.1: Remove columns ID and Y from the data if they are present
columns_to_drop = ['ID']
if 'y' in test_df.columns:
    columns_to_drop.append('y')

test_df = test_df.drop(columns_to_drop, axis=1)


In [16]:
# Step 7: Check for null and unique values for test and train sets
print("Train Data:")
print(train_df.isnull().sum())
print("\nUnique values in each column:")
print(train_df.nunique())

print("\nTest Data:")
print(test_df.isnull().sum())
print("\nUnique values in each column:")
print(test_df.nunique())

Train Data:
ID      0
y       0
X0      0
X1      0
X2      0
       ..
X380    0
X382    0
X383    0
X384    0
X385    0
Length: 378, dtype: int64

Unique values in each column:
ID      4209
y       2545
X0        47
X1        27
X2        44
        ... 
X380       2
X382       2
X383       2
X384       2
X385       2
Length: 378, dtype: int64

Test Data:
X0      0
X1      0
X2      0
X3      0
X4      0
       ..
X380    0
X382    0
X383    0
X384    0
X385    0
Length: 376, dtype: int64

Unique values in each column:
X0      49
X1      27
X2      45
X3       7
X4       4
        ..
X380     2
X382     2
X383     2
X384     2
X385     2
Length: 376, dtype: int64


In [17]:
# Step 8: If for any column(s), the variance is equal to zero, then you need to remove those variable(s).
zero_variance_cols = train_df.var()[train_df.var() == 0].index
train_df = train_df.drop(zero_variance_cols, axis=1)
test_df = test_df.drop(zero_variance_cols, axis=1)


<ipython-input-17-e3e5873df3f6>:2: FutureWarning: The default value of numeric_only in DataFrame.var is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  zero_variance_cols = train_df.var()[train_df.var() == 0].index
<ipython-input-17-e3e5873df3f6>:2: FutureWarning: The default value of numeric_only in DataFrame.var is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  zero_variance_cols = train_df.var()[train_df.var() == 0].index


In [21]:
# Step 8.1: Apply label encoder
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# Create a copy of the original data to avoid modifying the original
train_df_encoded = train_df.copy()
test_df_encoded = test_df.copy()

for column in train_df.select_dtypes(include=['object']).columns:
    # Fit label encoder on the training data and transform both train and test data
    train_df_encoded[column] = label_encoder.fit_transform(train_df[column])

    # Handle previously unseen labels in the test set
    test_df_encoded[column] = test_df[column].map(
        lambda s: label_encoder.transform([s])[0] if s in label_encoder.classes_ else -1
    )

# Check if there are still any object columns
print(train_df_encoded.dtypes[train_df_encoded.dtypes == 'object'])


Series([], dtype: object)


### Step 9 :
 This step is implicitly handled in the label encoding process. When we apply the label encoder to the 'X' columns in the dataset, it converts categorical values into numerical representations. So, by the end of Step 8.1, the data is already changed into numericals.

In [23]:
# Step 10: Perform dimensionality reduction
# Step 10.1: Linear dimensionality reduction using Singular Value Decomposition (SVD)
from sklearn.decomposition import TruncatedSVD


In [24]:
# Number of components for SVD
n_components = 20


In [35]:
# Step 10: Perform dimensionality reduction
# Step 10.1: Linear dimensionality reduction using Singular Value
# Decomposition of the data to project it to a lower dimensional space.
svd = TruncatedSVD(n_components=20, random_state=42)

# # Fit and transform the SVD on the train data
# train_svd = svd.fit_transform(train_df)
# # Transform the SVD on the test data
# test_svd = svd.transform(test_df)


In [ ]:
# Step 11: Training using xgboost
X_train, X_val, y_train, y_val = train_test_split(train_svd, y, test_size=0.2, random_state=42)

model = XGBRegressor(objective ='reg:squarederror')
model.fit(X_train, y_train)

# Step 12: Predict your test_df values using xgboost
y_pred = model.predict(X_val)

# Evaluate the model
mse = mean_squared_error(y_val, y_pred)
print(f"Mean Squared Error: {mse}")